 # Regex and Prodigy
 LAT

<font size= "3">
Annotation of entities for sentences based on prodigy's format for different labels for further usage in Machine learning. Prodigy is an annotation tool based on spaCy. Please see here:
    </p>
  <a href="https://prodi.gy/ "> Prodigy </a> 
<font 3>

## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import spacy 
import re
from prodigy.util import write_jsonl



In [2]:
pwd

'C:\\Projects_28_06\\notebooks\\yeghaneh\\BZML\\notebooks'

## Read the Data

In [3]:
importVersion = '013'

In [4]:
path= '../data/01_df_v{0}.pickle'.format(importVersion)# Put the path of the data in your local machine here, consider the letter "r" before the path
dfAstroNova = pd.read_pickle(path)

# Sort based on the Chapter 

In [5]:
# Sort the data based on the chapters of the book 
dfAstroNova['chapter'] = dfAstroNova.chapter.replace("appendix b",np.nan).astype(float)  
dfAstroNova = dfAstroNova.rename_axis('MyIdx').sort_values(by = ['chapter', 'MyIdx'], ascending = [True, True])
dfAstroNova.chapter.fillna('appendix b', inplace=True)

In [6]:
dfAstroNova.reset_index(inplace=True)
dfAstroNova=dfAstroNova.drop("MyIdx",axis=1,inplace=False)
dfAstroNova=dfAstroNova.drop("html",axis=1)

In [7]:
type(dfAstroNova)

pandas.core.frame.DataFrame

In [8]:
dfAstroNova.head(5)

,text,links,italic,chapter,graphic,table,marginal,sentences,tagged
0,Chapter 1,[],[],1,[],[],[],[Chapter 1],"[[(Chapter, None), (1, NUM)]]"
1,On the distinction between the first motion an...,[],[],1,[],[],[],[On the distinction between the first motion a...,"[[(On, None), (the, None), (distinction, None)..."
2,The testimony of the ages confirms that the mo...,[],[],1,[],[],[ Terms: 1. The first motion is that of the wh...,[The testimony of the ages confirms that the m...,"[[(The, None), (testimony, None), (of, None), ..."
3,It is just this from which astronomy arose amo...,[],[],1,[ ch 1 gr 1],[],[],[It is just this from which astronomy arose am...,"[[(It, None), (is, None), (just, None), (this,..."
4,Before the distinction between the first motio...,[],[(such],1,[],[],[ 2],[Before the distinction between the first moti...,"[[(Before, None), (the, None), (distinction, N..."


In [9]:
len(dfAstroNova)

1605

### Read the Sentences

In [10]:
df=dfAstroNova.sentences

In [11]:
sents=[]
for para in dfAstroNova.sentences:
    sents +=para

In [12]:
len(sents)

6699

## Annotation Based on Consistant format with Prodigy 

<fig size= 3>
Prodigy accept a specific format; a JSONL format (newline-delimited JSON). Entities and other highlighted spans of text can be defined in the "spans" property. A example could look like this dictionary:
</p>    
text': 'On 1595 October 30 at 8h 20m, they found Mars at 17° 48’ Taurus, with a diurnal motion of 22’ 54” ^15.',
  'spans': [{'start': 22, 'end': 29, 'label': 'TIME'}]}
</p>
which start and end's numbers refer to the position of entitiy  in the sentence. (here PARA)

    
<fig size>

In [15]:
def print_sents_entities(examples):
    for example in examples:
        if example["spans"]!=[]:
            sent,entity=example['text'], example['text'][example["spans"][0]["start"]:example["spans"][0]["end"]]
            print(sent,[entity])

In [14]:
print_sents_entities(examples)

NameError: name 'examples' is not defined

In [16]:
def print_entities(examples):
    entities = []
    for example in examples:
        if example["spans"] != []:
            entities += [example['text'][example["spans"][0]["start"]:example["spans"][0]["end"]]]
            len(entities)
    print("There is {} entities with this detail\n{}".format(len(entities),entities))

In [62]:
print_entities(examples)

There is 903 entities with this detail
['3° 4\' 52"', '41° 55\' 8"', '16° 33\' 30"', '7° 28\' 56"', '37° 32\' 37"', '4° 20\' 8"', '85° 39\' 52"', '11° 25\' 48"', '11° 23\' 53"', '5° 27’ 47”', '23° 32\' 16" Leo', '5° 27’ 47”', '5° 27’ 47”', '78° 51’ 54”', '47° 59’ 16”', '44° 59’ 10”', '44° 59’ 10”', '53° 26’ 17”', '44° 59’ 10”', '7° 30’ 10”', '3° 0’ 6”', '16° Cancer.', '16° Cancer', '1° 3’ 32” But', '2° 3’', '9° Leo.', '23° 45’ Leo.', '5° 2’', '23° 25’ Leo.', '23° 20’ Leo', '17° Taurus', '3° Cancer', '17° Cancer', '27° Sagittarius.', '26° 45’ 24” Gemini.', '26° 24’ 43” Sagittarius.', '26° 24’ 13” Sagittarius', '18° Taurus', '13° Pisces.', '18° Taurus', '17° Taurus', '4° 33’', '1° 50’', '8° 36’ 50” Gemini', '8° 20’ Gemini', '1° 55’', '1° 52’', '6° 41’ 50” Gemini', '6° 42’ 54”', '6° 46’ 10” Gemini.', '16° 47’ Cancer', '16° 46’ 16” Cancer', '21° 18’ 11” Leo', '21° 10’ 30” Leo', '25° 10’ 20” Virgo.', '25° 8’ 32” Virgo', '3° 58\' 21" Scorpio', '3° 55’ 49”', '3° 58’ 10”', '27° 15’ Sagittarius

## See Misaligned Tokens

<fig size= 3>
This step is very important, since if Pordigy faces with Misaligned tokens, we can see that before and try to adjust regex accordingly
<fig size>

In [20]:
def misaligned_token(examples):
    counter=0
    nlp = spacy.load("en_core_web_sm")  
    for example in examples:  
        doc = nlp(example["text"])
        for span in example["spans"]:
            char_span = doc.char_span(span["start"], span["end"])
            if char_span is None:  
                counter+=1
                print("{}- Misaligned tokens-->".format(counter), example["text"], span)

In [64]:
misaligned_token(examples)

1- Misaligned tokens--> On 1580 November 12 at 10h 50m,1 they set Mars down at 8° 36’ 50” Gemini2 without mentioning the horizontal variations,  by which term I wish the diurnal parallaxes and the refractions to be understood in what follows. {'start': 55, 'end': 72, 'label': 'LONG'}
2- Misaligned tokens--> On 1593 24 August at 10h 30m they report Mars as being at 12° 38’ Pisces12 with an observed diurnal motion of 16’ 45”, and this near the nonagesimal13 where there is no longitudinal parallax. {'start': 58, 'end': 72, 'label': 'LONG'}
3- Misaligned tokens--> But from this right ascension, I first select,  from Tycho’s table of right ascensions, the degree of the right sphere that was riding at the same time, 28° 1’ 0” Libra,27 whose declination, from another of that author’s tables,28 is 10° 48’ 30” [South]. {'start': 153, 'end': 168, 'label': 'LONG'}
4- Misaligned tokens--> Hence, its apparent longitude is found to be 0° 7’ Virgo,7 latitude 1° 36’ 6” N ^8. {'start': 45, 'end': 56, '

# Edge Cases

## Solution

We can modify our annotation rules based on the above examples

In [66]:
misaligned_token(examples)

1- Misaligned tokens--> On 1580 November 12 at 10h 50m,1 they set Mars down at 8° 36’ 50” Gemini2 without mentioning the horizontal variations,  by which term I wish the diurnal parallaxes and the refractions to be understood in what follows. {'start': 55, 'end': 72, 'label': 'LONG'}


# Refined Edge Cases

- 8° 36’ 50” Gemini2



### One Possible Solution 

In [68]:
print_entities(examples)

There is 1002 entities with this detail
['3° 4\' 52"', '41° 55\' 8"', '16° 33\' 30"', '7° 28\' 56"', '37° 32\' 37"', '1\' 33"', '4° 20\' 8"', '85° 39\' 52"', '11° 25\' 48"', '11° 23\' 53"', '1’ 55”', '5° 27’ 47”', '23° 32\' 16" Leo', '5° 27’ 47”', '5° 27’ 47”', '78° 51’ 54”', '4’ 24”', '4’ 24”', '47° 59’ 16”', '44° 59’ 10”', '44° 59’ 10”', '53° 26’ 17”', '44° 59’ 10”', '7° 30’ 10”', '3° 0’ 6”', '16° Cancer.', '16° Cancer', '1° 3’ 32” But', '2° 3’', '9° Leo.', '23° 45’', '5° 2’', '23° 25’', '23° 20’', '17° Taurus', '3° Cancer', '17° Cancer', '27° Sagittarius.', '26° 45’ 24” Gemini.', '26° 24’ 43” Sagittarius.', '10’ 20”', '26° 24’ 13” Sagittarius', '18° Taurus', '13° Pisces.', '18° Taurus', '17° Taurus', '4° 33’', '1° 50’', '8° 36’ 50” Gemini2', '8° 20’', '1° 55’', '1° 52’', '6° 41’ 50” Gemini', '1’ 4”', '6° 46’ 10” Gemini.', '16° 47’', '16° 46’ 16” Cancer', '21° 18’ 11” Leo', '24’ 15”', '7’ 41”', '21° 10’ 30” Leo', '25° 10’ 20” Virgo.', '1’ 48”', '25° 8’ 32” Virgo', '3° 58\' 21" Scorpi

In [69]:
misaligned_token(examples)

1- Misaligned tokens--> And δηζ will be 43° 26' 18", and δγζ 86° 52' 36", γδζ 46° 33' 42", and γδα 1° 7' 5"- a different angle from different beginnings. {'start': 75, 'end': 83, 'label': 'LONG'}
2- Misaligned tokens--> And for the first time the mean anomaly is 87° 9' 24", and 87° 16' 30"8 for the second. {'start': 59, 'end': 70, 'label': 'LONG'}
3- Misaligned tokens--> And in Chapter 28, at a mean anomaly of 11° 37', which, after the correction of Chapter 53 preceding, becomes 11° 52',2 the distance of the point on the ecliptic to which a perpendicular dropped from the body of Mars descends, was found to be 166,180, or 166,208. {'start': 110, 'end': 117, 'label': 'LONG'}
4- Misaligned tokens--> And since 2° 40’ 48" Pisces, the position on the orbit, is distant from 16° 43' Taurus10 by 74° 2', the inclination will therefore be 1° 46' 10". {'start': 72, 'end': 86, 'label': 'LONG'}
5- Misaligned tokens--> But when 36' are subtracted from Mars's aphelion, placing it at 0° 3' Leo26 for th

In [72]:
misaligned_token(examples)


1- Misaligned tokens--> On 1593 24 August at 10h 30m they report Mars as being at 12° 38’ Pisces12 with an observed diurnal motion of 16’ 45”, and this near the nonagesimal13 where there is no longitudinal parallax. {'start': 58, 'end': 72, 'label': 'LONG'}


In [76]:
misaligned_token(examples)

In [15]:
a_08=print_entities(examples)

NameError: name 'print_entities' is not defined

In [80]:
misaligned_token(examples)

1- Misaligned tokens--> Hence, owing to our removal of the vernal equinox westward from that star, which was then 28° 2½’,4 it is concluded that Mars’s apogee was at 23° 25’ Leo. {'start': 90, 'end': 97, 'label': 'LONG'}
2- Misaligned tokens--> In Tycho’s table of oppositions, the apparent latitude at 21° 16’ Leo was 4° 32⅙’22 {'start': 74, 'end': 81, 'label': 'LONG'}
3- Misaligned tokens--> The sun was at 22° 21’ 31“Aquarius. {'start': 15, 'end': 26, 'label': 'LONG'}
4- Misaligned tokens--> And since the latitude of the fixed star is 56° 22’, the remainder, by subtraction, is 4° 2½’—supposing that Mars was at precisely the same longitude as the fixed star. {'start': 87, 'end': 93, 'label': 'LONG'}
5- Misaligned tokens--> And on midnight of July 30 the position of Mars disagrees with Magini's midday position by 3° 25½’, and on August 10 by 3° 59½’,so that the difference is increased, but gradually less so. {'start': 120, 'end': 127, 'label': 'LONG'}


In [87]:
label = "LAT"   
texts = sents  

regex_patterns = [
                  re.compile(r"8° 36’ 50” Gemini2|87° 16\' 30\"8|16° 43\' Taurus10|0° 3\' Leo26|11° 52\',2|12° 38’ Pisces12"
                              "|\d{1,3}\s?°\s?\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”|“]\s?[A-Z][a-z.]+(,\d{1,2})?"
                              "|\d{1,3}\s?[\'|’|°]\s?\d{1,2}\s?[\'|’|\"|”|“]\s?\s?[A-Z][a-z.]+(,\d{1,2})?"
                              "|\d{1,3}\s?°\s?\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”|“]-?(,\d{1,2})?"
                              "|\d{1,3}\s?[\'|’|°]\s?\d{1,2}\s?[\'|’|\"|”|“](,\d{2})?"
                              "|\d{1,3}\s?°\s?[A-Z][a-z.]+(,\d{1,2})?")           
]     


regex_patterns = [
                  re.compile(r"\d{1,3}\s?°\s?\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\u00BC-\u00BE\u2150-\u215E]?[\"|”|“]\s?(N(?!\w)|S(?!\w)|southern|south|north)"
                              "|\d{1,3}\s?[\'|’|°]\s?\d{1,2}\s?[\u00BC-\u00BE\u2150-\u215E]?[\'|’|\"|”|“]\s?(N(?!\w)|S(?!\w)|southern|south|north)")
]

regex_patterns = [
                  re.compile(r"6° 6¼’ south,19"
                              "|\d{1,3}\s?°\s?\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\u00BC-\u00BE\u2150-\u215E]?[\"|”|“]\s?(southern|south|north)"
                              "|\d{1,3}\s?[\'|’|°]\s?\d{1,2}\s?[\u00BC-\u00BE\u2150-\u215E]?[\'|’|\"|”|“]\s?(southern|south|north)")
]
 
#(5° 20\' 2\"  Libra,|21° 44\' 56\"  Sagittarius,|(?<!\S)\d{1,2}\s?°\s\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”]\s[A-Z][a-z]+(?!\S?\S)|(?<!\S)\d{1,2}\s?°\s\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”].?(?!\S?\S)|(?<!\S)\d{1,2}\s?°\s?\d{1,2}\s?[\'|’]\s?[A-Z][a-z]+(?!\S?\S))[,:/.]?
examples = []
for text in texts:
    for expression in regex_patterns:
        spans = []
    for match in re.finditer(expression, text):
        start, end = match.span()
        span = {"start": start, "end": end, "label": label}
        spans.append(span)
    task = {"text": text, "spans": spans}
    examples.append(task)              

write_jsonl("NER_LATI_V02.jsonl", examples)



In [88]:
a_10=print_entities(examples)

There is 60 entities with this detail
['6° 26’ south', '5° 31’ south', '0° 5’ 15” north', '3’ 12” south', '0° 2’ south', '0° 0’ 45” north', '0° 2’ 20” south', '0° 0’ 25” north', '1° 7’ north', '1° 36’ 45” north', '13° 0’ 40” north', '1° 50⅔’ north', '1° 49⅔’ north', '1° 41⅔’ southern', '1° 52⅓’ south', '1° 36’ south', '2° 12⅖’ north', '1° 49¾’ north', '2° 10⅚’ south', '6° 3’ south', '1° 40’ north', '4° 6’ north', '4° 32’ 10“ north', '3° 38’ 16“ north', '3° 42’ 22“ north', '1° 4’ 20“ north', '1° 9’ 45“ north', '3° 55½’ south', '6° 5’ 30“ south', '5° 52’ 15“ south', '6° 2’ south', '0° 5’ 10“ north', '0° 8’ north', '3° 33’ north', '4° 8’ north', '4° 10⅔’ north', '2° 50’ north', '4° 12’ north', '4° 14’ north', '4° 7’ 40“ north', '31° 2½’ north', '2° 22’ north', "3° 23' north", '6° 6¼’ south', '1° 19\' 30" north', "17° 25' south", '1° 0\' 30" north', "2° 5' south", '1° 53½’ north', '1° 16⅔’ north', "1° 36' south", '6° 6¼’ south,19', '0° 6⅔’ north', '5° 14\' 30" south', '0° 1\' 15" south', '

In [89]:
misaligned_token(examples)

In [93]:
a_09=print_entities(examples)

There is 1096 entities with this detail
['3° 4\' 52"', '41° 55\' 8"', '16° 33\' 30"', '7° 28\' 56"', '37° 32\' 37"', '1\' 33"', '4° 20\' 8"', '85° 39\' 52"', '11° 25\' 48"', '11° 23\' 53"', '1’ 55”', '47° 59¼’', '5° 27’ 47”', '23° 32\' 16" Leo', '5° 27’ 47”', '5° 27’ 47”', '78° 51’ 54”', '4’ 24”', '4’ 24”', '47° 59’ 16”', '44° 59’ 10”', '44° 59’ 10”', '53° 26’ 17”', '44° 59’ 10”', '7° 30’ 10”', '3° 0’ 6”', '8½° Cancer', '16° Cancer', '1° 3’ 32” But', '2° 3’', '9° Leo.', '23° 45’ Leo.', '5° 2’', '28° 2½’,4', '23° 20’ Leo', '17° Taurus', '3° Cancer', '17° Cancer', '27° Sagittarius.', '26° 45’ 24” Gemini.', '26° 24’ 43” Sagittarius.', '10’ 20”', '26° 24’ 13” Sagittarius', '18° Taurus', '13° Pisces.', '18° Taurus', '17° Taurus', '4° 33’', '1° 50’', '8° 36’ 50” Gemini2', '8° 20’ Gemini', '1° 55’', '1° 52’', '6° 41’ 50” Gemini', '1’ 4”', '6° 46’ 10” Gemini.', '16° 47’ Cancer', '16° 46’ 16” Cancer', '21° 18’ 11” Leo', '24’ 15”', '7’ 41”', '21° 10’ 30” Leo', '25° 10’ 20” Virgo.', '1’ 48”', '25

## Try Regex on the observational Sentences

In [24]:
path= '../data/ObsSentsR01V05A02.pkl'# Put the path of the data in your local machine here, consider the letter "r" before the path
df = pd.read_pickle(path)

In [27]:
df.head()
df.sort_index()
df.reset_index(drop=True)

,Chapter,ParaIndex,SentIndex,Paragraph,Sentence,Entities,Label
0,10,218,967,"On 1580 November 12 at 10h 50m,1 they set Mars...","On 1580 November 12 at 10h 50m,1 they set Mars...","[[1580 November 12, DATE], [10h 50m,1 they, TI...",Obs
1,10,221,977,"On 1582 December 28 at 11h 30m, they set Mars ...","On 1582 December 28 at 11h 30m, they set Mars ...","[[1582 December 28, DATE], [11h 30m, TIME], [M...",Obs
2,10,222,984,"On 1585 January 31 at 12h 0m, Mars was placed ...","On 1585 January 31 at 12h 0m, Mars was placed ...","[[1585 January 31, DATE], [12h 0m, TIME], [Mar...",Obs
3,10,223,992,On 1587 March 7 at 19h 10m they deduced the po...,On 1587 March 7 at 19h 10m they deduced the po...,"[[1587 March 7, DATE], [19h 10m, TIME], [Mars,...",Obs
4,10,224,997,On 1589 April 15 at 12h 5m they established th...,On 1589 April 15 at 12h 5m they established th...,"[[1589 April 15, DATE], [12h 5m, TIME], [Mars,...",Obs
...,...,...,...,...,...,...,...
69,62,1412,5910,But since the position is rather far from the ...,On 1593 July 21 at 14h (in astronomical terms)...,"[[1593 July 21, DATE], [14h, TIME], [observed,...",Obs
70,65,1460,6116,And I was now about to choose the mean betwee...,For while the opposition was on the night foll...,"[[opposition, ASTR], [January 30, DATE], [lati...",Obs
71,69,1554,6538,Let us therefore follow whither our inclinati...,"Therefore, at Hven in the year of Christ 139 M...","[[Hven, NAME], [139, PARA], [May 27, DATE], [6...",Obs
72,69,1562,6554,"In the year of Christ 139 Pharmouthi 9, which ...","In the year of Christ 139 Pharmouthi 9, which ...","[[139, PARA], [February 23, DATE], [5h 30m, TI...",Obs


In [36]:
Obstexts =" ".join(list(df.Sentence))
Obstexts

'On 1580 November 12 at 10h 50m,1 they set Mars down at 8° 36’ 50” Gemini2 without mentioning the horizontal variations,  by which term I wish the diurnal parallaxes and the refractions to be understood in what follows. On 1582 December 28 at 11h 30m, they set Mars down at 16° 47’ Cancer by observation ^6. On 1585 January 31 at 12h 0m, Mars was placed at 21° 18’ 11” Leo ^8. On 1587 March 7 at 19h 10m they deduced the position of Mars from the observations, which was 25° 10’ 20” Virgo. On 1589 April 15 at 12h 5m they established the position of Mars very carefully at 3° 58\' 21" Scorpio, and corrected for longitudinal parallax so as to make it 3° 57’ 11” ^10. On 1591 June 6 at 12h 20m, Mars is placed at 27° 15’ Sagittarius ^11. On 1593 24 August at 10h 30m they report Mars as being at 12° 38’ Pisces12 with an observed diurnal motion of 16’ 45”, and this near the nonagesimal13 where there is no longitudinal parallax. On 1595 October 30 at 8h 20m, they found Mars at 17° 48’ Taurus, with a

In [37]:
label = "LAT"   
texts = sents  

regex_patterns = [
                  re.compile(r"8° 36’ 50” Gemini2|87° 16\' 30\"8|16° 43\' Taurus10|0° 3\' Leo26|11° 52\',2|12° 38’ Pisces12"
                              "|\d{1,3}\s?°\s?\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”|“]\s?[A-Z][a-z.]+(,\d{1,2})?"
                              "|\d{1,3}\s?[\'|’|°]\s?\d{1,2}\s?[\'|’|\"|”|“]\s?\s?[A-Z][a-z.]+(,\d{1,2})?"
                              "|\d{1,3}\s?°\s?\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”|“]-?(,\d{1,2})?"
                              "|\d{1,3}\s?[\'|’|°]\s?\d{1,2}\s?[\'|’|\"|”|“](,\d{2})?"
                              "|\d{1,3}\s?°\s?[A-Z][a-z.]+(,\d{1,2})?")           
]     


regex_patterns = [
                  re.compile(r"\d{1,3}\s?°\s?\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\u00BC-\u00BE\u2150-\u215E]?[\"|”|“]\s? (N|S|south|north))?"
                              "|\d{1,3}\s?[\'|’|°]\s?\d{1,2}\s?[\u00BC-\u00BE\u2150-\u215E]?[\'|’|\"|”|“]\s?(N|S|south|north)" 
]
 
#(5° 20\' 2\"  Libra,|21° 44\' 56\"  Sagittarius,|(?<!\S)\d{1,2}\s?°\s\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”]\s[A-Z][a-z]+(?!\S?\S)|(?<!\S)\d{1,2}\s?°\s\d{1,2}\s?[\'|’]\s?\d{1,2}\s?[\"|”].?(?!\S?\S)|(?<!\S)\d{1,2}\s?°\s?\d{1,2}\s?[\'|’]\s?[A-Z][a-z]+(?!\S?\S))[,:/.]?
examples = []
for text in Obstexts:
    for expression in regex_patterns:
        spans = []
    for match in re.finditer(expression, text):
        start, end = match.span()
        span = {"start": start, "end": end, "label": label}
        spans.append(span)
    task = {"text": text, "spans": spans}
    examples.append(task)              

#write_jsonl("NER_LATI_V02.jsonl", examples)


In [38]:
a_09=print_entities(examples)

There is 0 entities with this detail
[]


In [58]:
test="We shall now go through the observations1 again, carefully: On 1590 March 4 at 7h 10m, Mars was found by careful observation and calculation to be at 24° 22' 56\" Aries with latitude 0° 3' 20\" S. At that time, 8° Aries was setting, so Mars was rather low."

In [59]:
test[182:193]

'0° 3\' 20" S'

In [67]:
nlp = spacy.load("en_core_web_sm")  
doc = nlp(test)


In [68]:
doc

We shall now go through the observations1 again, carefully: On 1590 March 4 at 7h 10m, Mars was found by careful observation and calculation to be at 24° 22' 56" Aries with latitude 0° 3' 20" S. At that time, 8° Aries was setting, so Mars was rather low.

In [71]:
doc.char_span

<function Doc.char_span>

In [72]:
doc.ents

(1590 March 4, 7h 10, Mars, 24, 0, 8, Mars)